# Harassment and Newcomer Retention

In this notebook we investigate how receiving harassment correlates with newcomer activity and retention. For the purposes of this study, our measures of harassment are classifiers over individual discussion comments for personal attacks, aggression and toxicity. These classifiers were developed [in previous work](https://arxiv.org/abs/1610.08914). We will investigate the relationship between harassment and newcomer retention through running regression models that use a measures of editing activity and harassment in a first time span (t1) as independent variables and a measure of editing activity in a subsequent time span (t2) as the dependent variable.

In the following analyses, t1 is the first month after registering and t2 is the second month after registering, but this could be modified in further analysis. Our initial set of examples consists of all (20k) newcomers to English Wikipedia from 2001-2015 that received a personal attack or an aggressive or toxic comment on their user page by another user in t1 and a random sample of 100k newcomers from 2001-2015. We the reduced the set of examples to those newcomers who made at least one edit in t1. 

In [1]:
% matplotlib inline
import pandas as pd
from dateutil.relativedelta import relativedelta
import statsmodels.formula.api as sm
import requests

### Regression Modeling


In [2]:
#Features computes in ./Harassment and Newcomer Retention Data Munging.ipynb
df_reg = pd.read_csv("../../data/retention/newcomer_sample_features.csv")

In [3]:
def regress(df, formula, family = 'linear'):
    if family == 'linear':
        result = sm.ols(formula=f, data=df).fit()
    elif family == 'logistic':
        result = sm.logit(formula=f, data=df).fit(disp=0)
    else:
        print("Wrong Family")
    return result.summary().tables[1]

#### Logistic Regression: Does receiving harassment in t1 make you less likely to make an edit t2?

In [4]:
f = "t2_active ~ t1_harassment_received"
regress(df_reg, f, family = 'logistic')

,coef,std err,z,P>|z|,[95.0% Conf. Int.]
Intercept,-1.8661,0.009,-209.762,0.000,-1.884 -1.849
t1_harassment_received,0.8528,0.047,18.145,0.000,0.761 0.945


This model suggests that users who receive harassment have increased probability of being active in t2, compared to users who did not receive harassment. Lets control for how active the user was in t1 to see if the result holds.

In [5]:
f ="t2_active ~ t1_harassment_received + t1_num_days_active"
regress(df_reg, f, family = 'logistic')

,coef,std err,z,P>|z|,[95.0% Conf. Int.]
Intercept,-3.2661,0.016,-202.936,0.000,-3.298 -3.235
t1_harassment_received,-0.9343,0.077,-12.123,0.000,-1.085 -0.783
t1_num_days_active,0.4612,0.004,108.849,0.000,0.453 0.469


After controlling for the number of days a user was active in t1, we see that users receiving harassment have a decreased probability of activity in t2.

#### Linear Regression: Is harassment correlated with reduction in activity from t1 to t2?

Instead of running a logistic regression using an indicator for activity in t2 as our dependent variable, we will run a linear regression using the the number of days active in t2 as our dependent variable.

In [29]:
f ="t2_num_days_active ~ t1_harassment_received"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,1.0519,0.013,82.947,0.000,1.027 1.077
t1_harassment_received,0.9868,0.086,11.435,0.000,0.818 1.156


We see a similar results as above. Without controlling for activity in t1, users who receive harassment have, on average, more active days in t2.

In [30]:
f= "t2_num_days_active ~ t1_num_days_active + t1_harassment_received"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.9024,0.009,-102.129,0.000,-0.920 -0.885
t1_num_days_active,0.7959,0.002,441.331,0.000,0.792 0.799
t1_harassment_received,-2.0260,0.052,-38.602,0.000,-2.129 -1.923


However, when we control for the number of days a user is active in t1, we see that users who receive harassment have fewer active days in t2. The coefficient is significantly less than 0. Instead of using an indicator for whether the user received harassment, lets use the count of various types of harassment received (i.e personal attacks, aggression, toxicity)

In [31]:
f= "t2_num_days_active ~ t1_num_days_active + t1_num_attacks_received"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.9147,0.009,-103.327,0.000,-0.932 -0.897
t1_num_days_active,0.7923,0.002,439.183,0.000,0.789 0.796
t1_num_attacks_received,-1.3299,0.047,-28.116,0.000,-1.423 -1.237


In [32]:
f ="t2_num_days_active ~ t1_num_days_active + t1_num_aggression_received"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.9132,0.009,-103.197,0.000,-0.931 -0.896
t1_num_days_active,0.7932,0.002,439.481,0.000,0.790 0.797
t1_num_aggression_received,-1.2197,0.040,-30.231,0.000,-1.299 -1.141


In [33]:
f="t2_num_days_active ~ t1_num_days_active + t1_num_toxicity_received"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.9197,0.009,-103.745,0.000,-0.937 -0.902
t1_num_days_active,0.7900,0.002,437.905,0.000,0.786 0.794
t1_num_toxicity_received,-0.9688,0.049,-19.622,0.000,-1.066 -0.872


We see the same general pattern as above, accept that toxic comments received seem to have a weaker association with lower activity in t2 than personal attacks and aggression. Also, the magnitude of the coefficients decreased since we are using a count and not an indicator as above.

Before we regress an activity measure in t2 on an activity measure in t1 and multiple measures of harassment, lets see how our different measures of harassment correlate:

In [34]:
from scipy.stats import pearsonr 
print(pearsonr(df_reg['t1_num_attacks_received'] ,  df_reg['t1_num_aggression_received']))
print(pearsonr(df_reg['t1_num_toxicity_received'] , df_reg['t1_num_aggression_received']))
print(pearsonr(df_reg['t1_num_toxicity_received'] , df_reg['t1_num_attacks_received']))

(0.89652629461566002, 0.0)
(0.74724267108155895, 0.0)
(0.72947262995624695, 0.0)


Personal attacks and aggression are very highly correlated. This is probably because both questions appeared on the same form. The toxicity measure has a lower though still high correlation with both personal attacks and aggression. Let's try a regression using toxicity and one of the two other measures:

In [35]:
f="t2_num_days_active ~ t1_num_days_active + t1_num_toxicity_received + t1_num_attacks_received"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.9146,0.009,-103.317,0.000,-0.932 -0.897
t1_num_days_active,0.7923,0.002,439.109,0.000,0.789 0.796
t1_num_toxicity_received,0.0818,0.072,1.140,0.254,-0.059 0.223
t1_num_attacks_received,-1.3870,0.069,-20.134,0.000,-1.522 -1.252


This result is harder to interpret due to the strong correlation between `t1_num_toxicity_received` and `t1_num_attacks_received`.

#### How do gender, harassment and activity interact?

Gender in Wikipedia is not well defined. After registering, users have the ability to report their gender in their user preferences. The vast majority of users do not report their gender. This may be because reporting their gender is not important to them, they don't want to report a gender, or they simply are unaware of the feature. There is anecdotal evidence that users often report an incorrect gender. Overall, this means that we should expect users who report their gender to be different than the rest and we cannot be sure if reported genders are correct. Another caveat for the following analysis is that we do not know when the user reported their gender; they may have changed their user preference after our 2 month interval of interest.




In [36]:
f="t2_active ~ has_gender"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,0.1210,0.001,117.051,0.000,0.119 0.123
has_gender,0.3265,0.005,69.523,0.000,0.317 0.336


Users who supply a gender are also more likely to be active in t2, this is expected given how gender is reported.  Lets restrict to users with a gender and see if this is different for males and females:

In [37]:
f="t2_active ~ is_female"
regress(df_reg.query("has_gender == 1"), f, family = 'logistic')

,coef,std err,z,P>|z|,[95.0% Conf. Int.]
Intercept,-0.1510,0.029,-5.139,0.000,-0.209 -0.093
is_female,-0.4499,0.082,-5.457,0.000,-0.611 -0.288


Females have a decreased probability of being active in t1.

In [38]:
f="t1_harassment_received ~ has_gender"
regress(df_reg, f, family = 'logistic')

,coef,std err,z,P>|z|,[95.0% Conf. Int.]
Intercept,-3.8591,0.022,-178.597,0.000,-3.901 -3.817
has_gender,0.6822,0.073,9.379,0.000,0.540 0.825


Users who supply a gender are more likely to receive harassment! Lets see if this is different for males and females:

In [39]:
f="t1_harassment_received ~ is_female"
regress(df_reg.query("has_gender == 1"), f, family = 'logistic')

,coef,std err,z,P>|z|,[95.0% Conf. Int.]
Intercept,-3.3041,0.079,-41.683,0.000,-3.459 -3.149
is_female,0.7051,0.165,4.263,0.000,0.381 1.029


Females have an increased probability of receiving harassment in t1.

In [41]:
f="t2_num_days_active ~ t1_num_days_active + has_gender"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.9431,0.009,-106.206,0.000,-0.961 -0.926
t1_num_days_active,0.7760,0.002,421.985,0.000,0.772 0.780
has_gender,0.9819,0.036,27.110,0.000,0.911 1.053


Users who supply a gender appear to be more active in t2 even after controlling for activity in t1.

In [20]:
f="t2_num_days_active ~ t1_num_days_active + is_female"
regress(df_reg.query("has_gender == 1"), f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.5978,0.096,-6.212,0.000,-0.786 -0.409
t1_num_days_active,0.8843,0.009,100.709,0.000,0.867 0.901
is_female,-0.6060,0.204,-2.974,0.003,-1.006 -0.207


Females appear to have decreased activity in t2 even after controlling for activity in t1 compared to males.

In [21]:
f="t2_num_days_active ~ t1_num_days_active + t1_harassment_received * has_gender"
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.9256,0.009,-104.670,0.000,-0.943 -0.908
t1_num_days_active,0.7852,0.002,426.502,0.000,0.782 0.789
t1_harassment_received,-1.8914,0.055,-34.575,0.000,-1.999 -1.784
has_gender,1.0402,0.037,28.401,0.000,0.968 1.112
t1_harassment_received:has_gender,-1.5351,0.183,-8.395,0.000,-1.893 -1.177


It seems like users who supply a gender and receive harassment have even more strongly reduced activity in t2 compared to users who do not supply a gender and get harassed.

In [22]:
f="t2_num_days_active ~ t1_num_days_active + t1_harassment_received * is_female"
regress(df_reg.query("has_gender == 1"), f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.5329,0.096,-5.574,0.000,-0.720 -0.346
t1_num_days_active,0.8958,0.009,102.390,0.000,0.879 0.913
t1_harassment_received,-4.0475,0.404,-10.009,0.000,-4.840 -3.255
is_female,-0.4971,0.209,-2.384,0.017,-0.906 -0.088
t1_harassment_received:is_female,0.6461,0.839,0.770,0.441,-0.999 2.291


When restricting to users with a gender, we see that users with `t1_harassment_received` have reduced activity in t2, but the impact is not significantly different for males and females.

#### Addressing the bad newcomer confound

A serious potential confound in our analyses could be that the users who receive harassment are just bad faith newcomers or sock-puppets. They get attacked for their misbehavior and reduce their activity in t2 because they get blocked or because they never intended to stick around past their own attacks. To reduce this confound, we control for whether the user harassed anyone in t1 and for whether they received an user warning of any type:

In [23]:
f="t2_num_days_active ~ t1_num_days_active + t1_harassment_received + t1_harassment_made * t1_harassment_received + t1_num_warnings_recieved * t1_harassment_received "
regress(df_reg, f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.8925,0.009,-100.442,0.000,-0.910 -0.875
t1_num_days_active,0.8103,0.002,433.487,0.000,0.807 0.814
t1_harassment_received,-1.7093,0.062,-27.751,0.000,-1.830 -1.589
t1_harassment_made,-1.2465,0.053,-23.582,0.000,-1.350 -1.143
t1_harassment_made:t1_harassment_received,-0.1804,0.128,-1.410,0.158,-0.431 0.070
t1_num_warnings_recieved,-0.2231,0.020,-11.378,0.000,-0.262 -0.185
t1_num_warnings_recieved:t1_harassment_received,0.1221,0.042,2.893,0.004,0.039 0.205


Even users who receive harassment but did not harass anyone or receive a user warning show reduced activity in t2.

#### Harassment vs Previously identified newcomer socialization issues

[Halfak et al](https://www-users.cs.umn.edu/~halfak/publications/The_Rise_and_Decline/halfaker13rise-preprint.pdf) examine how user warnings and deletions and reverts correlate with newcomer retention. Here we add those features and see how they compare to measure of harassment.

In [28]:
f = "t2_num_days_active ~ t1_num_days_active +   t1_harassment_received   + t1_num_warnings_recieved +  t1_fraction_ns0_deleted + t1_fraction_ns0_reverted "
regress(df_reg.query("t1_num_ns0_edits > 0"), f)

,coef,std err,t,P>|t|,[95.0% Conf. Int.]
Intercept,-0.8718,0.016,-55.874,0.000,-0.902 -0.841
t1_num_days_active,0.8061,0.002,367.433,0.000,0.802 0.810
t1_harassment_received,-2.3466,0.071,-33.038,0.000,-2.486 -2.207
t1_num_warnings_recieved,-0.2076,0.022,-9.586,0.000,-0.250 -0.165
t1_fraction_ns0_deleted,-0.1547,0.062,-2.476,0.013,-0.277 -0.032
t1_fraction_ns0_reverted,-0.1563,0.026,-6.026,0.000,-0.207 -0.105


WIP: Receiving harassment is worse for a newcomer than receiving 11 warning messages or having all their first months work deleted or reverted.

### Investigate some newcomer experiences

Our regression analyses have established that newcomer who receive harassment show a greater subsequent decline in activity than normal. Let's look at a few example of newcomers, what edits they made and how other interacted with them.